# Interactive charts for understanding volatility

### By Saeed Amen (@thalesians) - Managing Director & Co-founder of the Thalesians

Changing the sample frequency for computing realised volatility will change final result. In this study, we show how we can create an interactive study for users to change the sampling frequency and period window to see how realised volatility changes.

We can do this using Dash which can be downloaded [here](https://github.com/chriddyp/dash/). Dash makes it easy to create Flask, JS, and CSS boilerplate for interactive, web-based visualization apps in Python. Also take a look at [this](http://moderndata.plot.ly/create-a-plotly-dashboards-in-under-10-minutes/) which shows how to setup your own cloud instance.

We shall be using Python, together with Plotly for plotting. Plotly is a free web-based platform for making graphs. You can keep graphs private, make them public, and run Plotly on your [Plotly Enterprise on your own servers](https://plot.ly/product/enterprise/). You can find more details [here](https://plot.ly/python/getting-started/).

We shall be using market data from Bloomberg (using Brian Smith's TIA wrapper). We have also written the code to allow importation of market data from CSV files (for example if you've like to use intraday data from an FX broker) and other online sources. For more information on how to access Bloomberg via Python, please take a look at [this](https://plot.ly/ipython-notebooks/ukelectionbbg/), where I discussed how to use Bloomberg data to do an event study for price action around UK general elections. I have reused some of the code from that earlier project and also added a few extra features to some of that code.

### Python scripts

* bbg_com - low level interaction with BBG COM object (adapted for Python 3.4) (which we are simply calling)
* datadownloader - wrapper for BBG COM, Quandl, Yahoo and CSV access to data
* realisedvolslider - interactive analysis of realised volatility

### Downloading the market data

We create the DataDownloader class, which acts a wrapper for various market data sources. The idea is to decouple the precise vendor implementations with our higher level code. This code can be used to download historic market data. For daily historic data we use "download_time_series" which has support Bloomberg, Quandl, Yahoo and CSV data sources. For intraday historic data, Bloomberg and CSV data sources are implemented. Obviously, if your CSV files have different date formats, you can edit the code to reflect them.

In [1]:
# for time series manipulation
import pandas
import pandas.io.data as web

import datetime

class DataDownloader:    
    def download_time_series(self, vendor_ticker, pretty_ticker, start_date, source, csv_file = None,
                             freq = 'daily', freq_no = 1):
        if not(isinstance(start_date, list)):
            start_date = [start_date]

        if freq == 'daily':
            if source == 'Quandl':
                import Quandl
                # Quandl requires API key for large number of daily downloads
                # https://www.quandl.com/help/api
                spot = Quandl.get(vendor_ticker)
                spot = pandas.DataFrame(data = spot['Value'], index = spot.index)
                spot.columns = [pretty_ticker]
            elif source == 'Yahoo':
                finish_date = datetime.datetime.utcnow()
                finish_date = datetime.datetime(finish_date.year, finish_date.month, finish_date.day, 0, 0, 0)

                spot = web.DataReader(vendor_ticker, 'yahoo', start_date[0], finish_date)
                spot = pandas.DataFrame(data = spot['Close'].values, index = spot.index, columns = [pretty_ticker])

                spot.index = pandas.DatetimeIndex(spot.index)

            elif source == 'Bloomberg':
                from egthalesians.plotly.helper.bbg_com import HistoricalDataRequest
                req = HistoricalDataRequest([vendor_ticker], ['PX_LAST'], start = start_date[0])
                req.execute()

                spot = req.response_as_single()
                spot.columns = [pretty_ticker]
            elif source == 'CSV':
                dateparse = lambda x: pandas.datetime.strptime(x, '%Y-%m-%d')

                # in case you want to use a source other than Bloomberg/Quandl
                spot = pandas.read_csv(csv_file, index_col=0, parse_dates=0, date_parser=dateparse)

        elif freq == 'intraday':
            if source == 'Bloomberg':
                from bbg_com import IntrdayBarRequest
                req = IntrdayBarRequest(vendor_ticker, freq_no, start = start_date[0])

                req.execute()

                spot = req.response
                spot.columns = [pretty_ticker + '.' + x for x in spot.columns]

                spot = pandas.DataFrame(data = spot[pretty_ticker + ".close"].values,
                                        index = spot.index, columns = [pretty_ticker + ".close"])

            elif source == 'CSV':
                dateparse = lambda x: pandas.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

                # in case you want to use a source other than Bloomberg/Quandl etc
                try:
                    spot = pandas.read_csv(csv_file, index_col = 0, parse_dates = 0, date_parser = dateparse)
                except:
                    dateparse = lambda x: pandas.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')
                    spot = pandas.read_csv(csv_file, index_col = 0, parse_dates = 0, date_parser = dateparse)

        return spot
    
        DataDownloader.download_time_series = download_time_series

### Interactive volatility study

The first step is to import all the necessarily modules, in particular note the use of Flash and Dash. We also import various libraries like Pandas which are necessary for time series manipulation.

In [2]:
# for time series/maths
import pandas
import math
import datetime
from datetime import timedelta

# Flask application
from flask import Flask, render_template
from flask.ext.socketio import SocketIO, emit

# for plotting
import json
import plotly

# Dash components
import dash.utils as utils
from dash.components import element as el
from dash.components import graph

We create a wrapper function to download EUR/USD spot data, that is needed for our realised volatility calculation. Users can choose either to use Bloomberg or a CSV file.

In [3]:
def load_data():
    ticker = 'EURUSD' # will use in plot titles later (and for creating Plotly URL)

    ##### download intraday EUR/USD data from Bloomberg or CSV file
    # source = "Bloomberg"
    source = "CSV"

    csv_file = None

    data_downloader = DataDownloader()
    start_date = datetime.datetime.utcnow() - timedelta(days = 120)
    freq = 'intraday'

    if source == 'Bloomberg':
        vendor_ticker = 'EURUSD BGN Curncy'
    elif source == 'CSV':
        # you can get free FX intraday data from Gain Capital (if you don't have Bloomberg)
        vendor_ticker = 'EURUSD'
        csv_file = 'EURUSD.csv'

    return data_downloader.download_time_series(vendor_ticker, ticker, start_date, source, csv_file = csv_file, freq = freq)

We define the general properties for the Flask application. We also define the HTML page, including the header, various sliders and a text box. Users will be able to change the "period" slider to change the window used for the volatility computation. Similarly, they can change the "frequency" slider to change the sampling frequency. Users can type in a plot title into the text box.

In [4]:
# define properties for application
name = 'dash-realised-vol-slide'
app = Flask(name)
app.debug = True
app.config['key'] = 'secret'
socketio = SocketIO(app)
spot = load_data()
graph_id = 'realised-vol-slider'

# write the HTML "includes" blocks to /templates/runtime/dash-1-hello-world
# alternatively, include the HTML yourself in that folder
utils.write_templates(
    {
        'header': [
            el('H1', {}, 'Dash to investigate realised vol')
        ],

        'controls': [
            el('label', {}, 'Frequency 5 to 60 mins'),
            el('input', {
                'type': 'range',
                'class': 'u-full-width show-values',
                'name': 'frequency',
                'value': 0,
                'min': 5,
                'max': 60,
                'step': 5
            }),
            el('label', {}, 'Period 1 to 20 days'),
            el('input', {
                    'type': 'range',
                    'class': 'u-full-width show-values',
                    'name': 'period',
                    'value': 0,
                    'min': 1,
                    'max': 20,
                    'step': 1
                }),
            el('label', {}, 'Title'),
            el('input', {
                'type': 'text',
                'name': 'title',
                'placeholder': 'Type away',
                'class': 'u-full-width'
            }, '')
        ],

        'main_pane': [
            graph(graph_id)
        ]
    }, name
)

Specify the default HTML file to use for our rendering. We have used one included as a template in Dash.

In [5]:
@app.route('/')
def index():
    return render_template('layouts/layout_single_column_and_controls.html',
                           app_name = name)

This function will be called each time the user changes the values of the sliders. It recalculates the realised volatility dependent on the specified "period" and "frequency" sliders. It is then pushed to Plotly as JSON.

In [6]:
@socketio.on('replot')
def replot(app_state):
    print(app_state)
    frequency = float(app_state['frequency'])
    period = float(app_state['period'])
    period_mins = period * (1440.0 / frequency)

    # resample data for minute frequency specified
    resampled = spot.loc[spot.index.minute % frequency == 0]
    resampled = resampled.dropna()

    # calculate returns (need for realised volatility calculation)
    rets = resampled / resampled.shift(1) - 1

    # calculate realised volatility (careful with annualisation factor!)
    vol = pandas.rolling_std(rets, period_mins) * math.sqrt(252.0 * (1440.0 / frequency)) * 100

    # resample to 60 minute data (quicker to plot)
    to_plot = vol.loc[vol.index.minute % 60 == 0]

    x = pandas.to_datetime(to_plot.index.values, format='%Y-%m-%d %H:%M')
    y = to_plot.ix[:,0]

    # define graph in JSON format
    messages = [
        {
            'id': graph_id,
            'task': 'newPlot',
            'data': [{
                'x': x,
                'y': y
            }],
            'layout': {
                'xaxis': {
                    'title' : 'Date'
                },
                'yaxis': {
                    'title' : 'Vol'
                },
                'title': app_state.get('title', '') + 'freq = ' + str(frequency) + ' mins, period = '
                         + str(period) + ' days, ' + str(period_mins) + ' mins'
            }
        }
    ]

    # post JSON message
    emit('postMessage', json.dumps(messages, cls = plotly.utils.PlotlyJSONEncoder))

If we were running this in practice, we need to kick off the Flask based webserver. The below line of code does this (we have commented this out, because we need to do this outside the jupyter notebook).

In [7]:
# socketio.run(app)

We can see the app in action below in an animated gif.

In [8]:
from IPython.display import Image
Image(url='http://imgur.com/p1anUG4.gif', width=750)

### Biography

Saeed Amen is the managing director and co-founder of the Thalesians. He has a decade of experience creating and successfully running systematic trading models at Lehman Brothers, Nomura and now at the Thalesians. Independently, he runs a systematic trading model with proprietary capital. He is the author of Trading Thalesians – What the ancient world can teach us about trading today (Palgrave Macmillan). He graduated with a first class honours master’s degree from Imperial College in Mathematics & Computer Science. He is also a fan of Python and has written an extensive library for financial market backtesting called PyThalesians, which is partially open sourced - available on the [Thalesians GitHub page](https://github.com/thalesians)

Follow the Thalesians on Twitter @thalesians and get my book on Amazon [here](http://www.amazon.co.uk/Trading-Thalesians-Saeed-Amen/dp/113739952X). You can also join our Thalesians Meetup.com group [here](http://www.meetup.com/thalesians) - we do quant finance events in a number of cities including London, New York, Budapest, Prague and Frankfurt.

The Thalesians website can be found [here](http://www.thalesians.com).